['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장/총자본',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장/총자본', '당기순이익/총자본', '매출채권/총자본',
       '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본', '판매 및 관리비/총자본',
       '타법인주식및출자증권취득결정자기자본대비_전체시장+차입금의존도',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '거래량회전율', '최대주주비율']

* scaler : rank사용   / k=4 / [Q로 민 경우] 라벨 3개의 일치도 0.8648648648648649  // [Y로 민 경우] 라벨 3개의 일치도 1.0
* scaler : minmax사용 / k=4 / [Q로 민 경우] 라벨 3개의 일치도 0.8108108108108109 // [Y로 민 경우] 라벨 3개의 일치도 0.8 

In [1]:
import os
import numpy as np
import pandas as pd 
from krx_fr.preprocess import tools, scaler
from krx_fr.cluster.optimize import Optimizer
from krx_fr.cluster.kmeans import MyKmeans

In [2]:
# 윈도우는 Y로 고정
window_size = "Y"

# 어떻게 윈도우를 밀건지
freq = "Y"

# 클러스터 수
num_k = 4

sca = scaler.rank

kospi_listed = tools.load_json("..\\final_data\\코스피_구성종목.json")
kosdaq_listed = tools.load_json("..\\final_data\\코스닥_구성종목.json")
together_listed = tools.aggregate_listed_code(kospi_listed, kosdaq_listed)

In [3]:
# 퐅더에서 row_data 불러오기
kind_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\KIND\\")
financial_data = tools.read_data_from_folder("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\재무\\")

# label_data 불러오기
label_data_war = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\투자주의환기종목지정카운트_전체시장.csv", index_col=0)
label_data_man= pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\관리종목카운트_전체시장.csv", index_col=0)
label_data_emb = pd.read_csv("C:\\Jehyeon\\2022_Winter_KRX\\KRX\\final_data\\label\\횡령배임혐의발생카운트_전체시장.csv", index_col=0)

# 데이터처리 sum / mean
kind_data2 = tools.dict_data_preprocess_upgrade(kind_data.copy(), function=np.sum, window_size=window_size,rolling_size=freq, fillna=True)
financial_data2 = tools.dict_data_preprocess_upgrade(financial_data.copy(), function=np.mean, window_size=window_size, rolling_size=freq,fillna=False)

In [4]:
# 재무데이터와 공시데이터 합치기
concat_data = tools.dict_data_merge(kind_data2.copy(),financial_data2.copy(), dropna=False)

# 시기별 전체시장
#concat_together = tools.dict_data_match_listed(concat_data.copy(), together_listed, "Y")
# 시기별 코스피
#concat_kospi = tools.dict_data_match_listed(concat_data.copy(), kospi_listed, "Y")
# 시기별 코스닥
#concat_kosdaq = tools.dict_data_match_listed(concat_data.copy(), kosdaq_listed, "Y")

In [5]:
# 컬럼을 더할 경우
concat_together1 = tools.dict_data_plus(concat_data.copy(),
                                  plus_one="BW금액_전체시장"
                                 ,plus_two="CB금액_전체시장", return_origianl=False)
#concat_kospi1 = tools.dict_data_plus(concat_kospi,
#                                  plus_one="BW금액_전체시장"
#                                 ,plus_two="CB금액_전체시장", return_origianl=False)
#concat_kosdaq1 = tools.dict_data_plus(concat_kosdaq,
#                                  plus_one="BW금액_전체시장"
#                                 ,plus_two="CB금액_전체시장", return_origianl=False)
#
concat_together1["2012-01"].columns

Index(['BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장', '불성실공시법인지정카운트_전체시장',
       '상호변경카운트_전체시장', '업종변경카운트_전체시장', '유상증자금액_전체시장', '자기주식처분결정금액_전체시장',
       '조회공시요구카운트_전체시장', '최대주주변경을수반하는주식담보제공금액_전체시장', '최대주주변경을수반하는주식양수도금액_전체시장',
       '최대주주변경카운트_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장', '특수관계인에대한자금대여금액_전체시장',
       '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본',
       '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [6]:
concat_together2 = tools.dict_data_plus(concat_together1, 
                                  plus_one="상호변경카운트_전체시장"
                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kospi2 = tools.dict_data_plus(concat_kospi1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq2 = tools.dict_data_plus(concat_kosdaq1, 
#                                  plus_one="상호변경카운트_전체시장"
#                                 ,plus_two="업종변경카운트_전체시장", return_origianl=False)
concat_together2["2012-01"].columns

Index(['상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장',
       '불성실공시법인지정카운트_전체시장', '유상증자금액_전체시장', '자기주식처분결정금액_전체시장', '조회공시요구카운트_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장', '최대주주변경을수반하는주식양수도금액_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '특수관계인에대한자금대여금액_전체시장', '거래량회전율', '당기순이익',
       '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비'],
      dtype='object')

In [7]:
concat_together3 = tools.dict_data_plus(concat_together2, 
                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kospi3 = tools.dict_data_plus(concat_kospi2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq3 = tools.dict_data_plus(concat_kosdaq2, 
#                                  plus_one="최대주주변경을수반하는주식담보제공금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#
concat_together3["2012-01"].columns

Index(['최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장',
       '불성실공시법인지정카운트_전체시장', '유상증자금액_전체시장', '자기주식처분결정금액_전체시장', '조회공시요구카운트_전체시장',
       '최대주주변경카운트_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장', '특수관계인에대한자금대여금액_전체시장',
       '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본',
       '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [8]:
concat_together4 = tools.dict_data_plus(concat_together3, 
                                  plus_one="자기주식처분결정금액_전체시장"
                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kospi4 = tools.dict_data_plus(concat_kospi3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
#concat_kosdaq4 = tools.dict_data_plus(concat_kosdaq3, 
#                                  plus_one="자기주식처분결정금액_전체시장"
#                                 ,plus_two="특수관계인에대한자금대여금액_전체시장", return_origianl=False)
concat_together4["2012-01"].columns

Index(['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장',
       '불성실공시법인지정카운트_전체시장', '유상증자금액_전체시장', '조회공시요구카운트_전체시장', '최대주주변경카운트_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익',
       '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [9]:
concat_together5 = tools.dict_data_plus(concat_together4,
                                  plus_one="불성실공시법인지정카운트_전체시장"
                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kospi5 = tools.dict_data_plus(concat_kospi4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq5 = tools.dict_data_plus(concat_kosdaq4,
#                                  plus_one="불성실공시법인지정카운트_전체시장"
#                                 ,plus_two="조회공시요구카운트_전체시장", return_origianl=False)
concat_together5["2012-01"].columns

Index(['불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '경영권분쟁소송_전체시장',
       '유상증자금액_전체시장', '최대주주변경카운트_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비'],
      dtype='object')

In [10]:
concat_together6 = tools.dict_data_plus(concat_together5,
                                  plus_one="경영권분쟁소송_전체시장"
                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kospi6 = tools.dict_data_plus(concat_kospi5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
#concat_kosdaq6 = tools.dict_data_plus(concat_kosdaq5,
#                                  plus_one="경영권분쟁소송_전체시장"
#                                 ,plus_two="최대주주변경카운트_전체시장", return_origianl=False)
concat_together6["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장', '불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', 'BW금액_전체시장+CB금액_전체시장', '유상증자금액_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익',
       '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [11]:
concat_together7 = tools.dict_data_plus(concat_together6,
                                  plus_one="BW금액_전체시장+CB금액_전체시장"
                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kospi7 = tools.dict_data_plus(concat_kospi6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
#concat_kosdaq7 = tools.dict_data_plus(concat_kosdaq6,
#                                  plus_one="BW금액_전체시장+CB금액_전체시장"
#                                 ,plus_two="유상증자금액_전체시장", return_origianl=False)
concat_together7["2012-01"].columns

Index(['BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장',
       '불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '상호변경카운트_전체시장+업종변경카운트_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율',
       '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본', '최대주주비율',
       '판매 및 관리비'],
      dtype='object')

In [12]:
concat_together8 = tools.dict_data_plus(concat_together7,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kospi8 = tools.dict_data_plus(concat_kospi7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
#concat_kosdaq8 = tools.dict_data_plus(concat_kosdaq7,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장"
#                                 ,plus_two="상호변경카운트_전체시장+업종변경카운트_전체시장", return_origianl=False)
concat_together8["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장',
       '최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익',
       '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [13]:
concat_together9 = tools.dict_data_plus(concat_together8,
                                  plus_one="자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장"
                                 ,plus_two="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kospi9 = tools.dict_data_plus(concat_kospi8,
#                                  plus_one="자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
#concat_kosdaq9 = tools.dict_data_plus(concat_kosdaq8,
#                                  plus_one="자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장"
#                                 ,plus_two="최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장", return_origianl=False)
concat_together9["2012-01"].columns

Index(['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '타법인주식및출자증권취득결정자기자본대비_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익',
       '영업현금흐름', '차입금의존도', '총자본', '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [14]:
concat_together10 = tools.dict_data_plus(concat_together9,
                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장"
                                 ,plus_two="불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장", return_origianl=False)
#concat_kospi10 = tools.dict_data_plus(concat_kospi9,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장"
#                                 ,plus_two="불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장", return_origianl=False)
#concat_kosdaq10 = tools.dict_data_plus(concat_kosdaq9,
#                                  plus_one="경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장"
#                                 ,plus_two="불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장", return_origianl=False)
concat_together10["2012-01"].columns

Index(['경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '타법인주식및출자증권취득결정자기자본대비_전체시장',
       '거래량회전율', '당기순이익', '매출채권', '매출총이익', '영업이익', '영업현금흐름', '차입금의존도', '총자본',
       '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [15]:
concat_together11 = tools.dict_data_plus(concat_together10,
                                  plus_one="타법인주식및출자증권취득결정자기자본대비_전체시장"
                                 ,plus_two="차입금의존도", return_origianl=False)
#concat_kospi11 = tools.dict_data_plus(concat_kospi10,
#                                  plus_one="타법인주식및출자증권취득결정자기자본대비_전체시장"
#                                 ,plus_two="차입금의존도", return_origianl=False)
#concat_kosdaq11 = tools.dict_data_plus(concat_kosdaq10,
#                                  plus_one="타법인주식및출자증권취득결정자기자본대비_전체시장"
#                                 ,plus_two="차입금의존도", return_origianl=False)
concat_together11["2012-01"].columns

Index(['타법인주식및출자증권취득결정자기자본대비_전체시장+차입금의존도',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장', '거래량회전율', '당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '총자본', '최대주주비율', '판매 및 관리비'],
      dtype='object')

In [16]:
divide_together = tools.dict_data_divide(concat_together11.copy(),divide_what=['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장','당기순이익', '매출채권', '매출총이익',
       '영업이익', '영업현금흐름', '판매 및 관리비'], divide_into="총자본", return_origianl=False)
#divide_kospi = tools.dict_data_divide(concat_kospi11,divide_what=['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
#       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장','당기순이익', '매출채권', '매출총이익',
#       '영업이익', '영업현금흐름', '판매 및 관리비'], divide_into="총자본", return_origianl=False)
#divide_kosdaq = tools.dict_data_divide(concat_kosdaq11,divide_what=['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장',
#       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장','당기순이익', '매출채권', '매출총이익',
#       '영업이익', '영업현금흐름', '판매 및 관리비'], divide_into="총자본", return_origianl=False)

print(len(divide_together["2012-01"].columns))
divide_together["2012-01"].columns

12


Index(['자기주식처분결정금액_전체시장+특수관계인에대한자금대여금액_전체시장+최대주주변경을수반하는주식담보제공금액_전체시장+최대주주변경을수반하는주식양수도금액_전체시장/총자본',
       'BW금액_전체시장+CB금액_전체시장+유상증자금액_전체시장/총자본', '당기순이익/총자본', '매출채권/총자본',
       '매출총이익/총자본', '영업이익/총자본', '영업현금흐름/총자본', '판매 및 관리비/총자본',
       '타법인주식및출자증권취득결정자기자본대비_전체시장+차입금의존도',
       '경영권분쟁소송_전체시장+최대주주변경카운트_전체시장+상호변경카운트_전체시장+업종변경카운트_전체시장+불성실공시법인지정카운트_전체시장+조회공시요구카운트_전체시장',
       '거래량회전율', '최대주주비율'],
      dtype='object')

In [17]:
drop_togehter = tools.dict_data_drop(divide_together.copy(), dropna=True)
scaled_together = tools.dict_data_scale(drop_togehter.copy(), scaler=sca)

#drop_kospi = tools.dict_data_drop(divide_kospi, dropna=True)
#scaled_kospi = tools.dict_data_scale(drop_kospi, scaler=scaler.rank)

#drop_kosdaq = tools.dict_data_drop(divide_kosdaq, dropna=True)
#scaled_kosdaq = tools.dict_data_scale(drop_kosdaq, scaler=scaler.rank)

In [18]:
# Run_Kmeans
model_save_together = {}
for time, dataframe in scaled_together.items():
    cluster_model = MyKmeans(dataframe)
    cluster_model.set_params()
    #cluster_model.find_optimal_k(max_k=10, max_sample=10, optimize_method="silhouette")
    cluster_model.find_optimal_initp(num_of_cluster= num_k, max_sample=50, optimize_method="silhouette")
    k_mean_dict = cluster_model.run_kmean(num_of_cluster=num_k)
    model_save_together[time] = k_mean_dict

# 코스피
#model_save_kospi = {}
#for time, dataframe in scaled_kospi.items():
#    cluster_model = MyKmeans(dataframe)
#    cluster_model.set_params()
#    #cluster_model.find_optimal_k(max_k=10, max_sample=10, optimize_method="silhouette")
#    cluster_model.find_optimal_initp(4, max_sample=50, optimize_method="silhouette")
#    k_mean_dict = cluster_model.run_kmean(4)
#    model_save_kospi[time] = k_mean_dict
#
## 코스닥
#model_save_kosdaq = {}
#for time, dataframe in scaled_kosdaq.items():
#    cluster_model = MyKmeans(dataframe)
#    cluster_model.set_params()
#    #cluster_model.find_optimal_k(max_k=10, max_sample=10, optimize_method="silhouette")
#    cluster_model.find_optimal_initp(4, max_sample=50, optimize_method="silhouette")
#    k_mean_dict = cluster_model.run_kmean(4)
#    model_save_kosdaq[time] = k_mean_dict

100%|██████████| 50/50 [00:03<00:00, 13.50it/s]


In [19]:
# 레이블 데이터 shift시켜주기
label_data_man_shift = tools.shift_label_data(label_data_man.copy(),shift_size='Y')
label_data_war_shift = tools.shift_label_data(label_data_war.copy(),shift_size='Y')
label_data_emb_shift = tools.shift_label_data(label_data_emb.copy(),shift_size='Y')

label_data_pp1 = tools.dict_data_preprocess_upgrade(label_data_man_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_man = tools.dict_data_scale(label_data_pp1.copy(), scaler.zero_one)

label_data_pp2 = tools.dict_data_preprocess_upgrade(label_data_war_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_war = tools.dict_data_scale(label_data_pp2.copy(), scaler.zero_one)

label_data_pp3 = tools.dict_data_preprocess_upgrade(label_data_emb_shift.copy(), np.sum, "Y", rolling_size=freq)
label_data_pp_emb = tools.dict_data_scale(label_data_pp3.copy(), scaler.zero_one)

In [20]:
def eval_cluster_size(model_save:dict):
    return_df = pd.DataFrame()
    for t, model_dict in model_save.items():
        a = pd.Series(model_save[t]["model"].labels_).value_counts()
        a = pd.DataFrame(a)
        a.columns = [t]
        return_df = pd.concat([return_df, a], axis=1)
    return_df.loc["기업수",:] = return_df.sum(axis=0)
    return return_df
    
def eval_cluster_result(model_save:dict, raw_data:dict, label_data:dict, freq=freq, return_best_cluster=True):
    freq = freq + "S"
    date_range = pd.date_range("2012-01-01","2021-01-01", freq=freq)
    return_df = pd.DataFrame()
    for date in date_range:
        date = str(date)[:7]
        eval = Optimizer.eval(model_save[date]["model"], raw_data[date], label_data[date])
        df = pd.DataFrame(eval.values(), eval.keys(), columns=[date])
        return_df = pd.concat([return_df,df], axis=1)
    if return_best_cluster:
        return_df.loc["포착군집", :] = np.argmax(return_df.values, axis=0)
    return return_df

def eval_cluster_distance(prob_df, raw_data):
    ser = prob_df.loc["포착군집"].copy()
    tmp_dict = {}
    return_dict = {}
    for t,danger_cluster in ser.iteritems(): #포착군집을 반복한다
        t = str(t)
        num = int(danger_cluster)
        distance = cluster_model.distance_decomposition(model_save_together[t]["model"])[num] #최대확률 클러스터와 타 클러스터의 거리
        tmp_dict[t] = distance # 시기:거리

    for t, distance_matrix in tmp_dict.items():
        distance_sum = np.zeros(raw_data[t].columns.shape, dtype='float64')
        for i in range(len(distance_matrix)):
            slice = distance_matrix[i]
            if len(slice) ==0:
                continue
            distance_sum += slice
        return_dict[t] = distance_sum
        col = raw_data[t].columns
    
    return_df = pd.DataFrame(return_dict).T
    return_df.columns = col
    return_df_sq = return_df.copy()**2
    sum_square = return_df_sq.sum(axis=1)
    
    
    return return_df

In [22]:
# 이전의 Model로 이후의 데이터 클러스터링을 평가
#Optimizer.eval(model_save_together["2021-01"]["model"], scaled_together["2022-01"], label_data_pp1["2022-01"])

# 예측
#pd.DataFrame(model_save_together["2021-01"]["model"].predict(scaled_together["2022-07"])).value_counts()

### 전체 기업 평가

In [23]:
pd.set_option('display.max_columns', 130)

In [24]:
eval_cluster_size(model_save_together)#.max(axis=1)

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01,2022-01
0,463.0,361.0,382.0,394.0,489.0,587.0,457.0,513.0,439.0,491.0,480.0
1,450.0,408.0,445.0,489.0,437.0,481.0,583.0,510.0,584.0,606.0,544.0
3,418.0,455.0,443.0,466.0,411.0,393.0,375.0,556.0,534.0,502.0,541.0
2,339.0,444.0,426.0,408.0,474.0,435.0,546.0,431.0,515.0,522.0,566.0
기업수,1670.0,1668.0,1696.0,1757.0,1811.0,1896.0,1961.0,2010.0,2072.0,2121.0,2131.0


In [30]:
eval_one = eval_cluster_result(model_save_together, raw_data = scaled_together, label_data=label_data_pp1)
eval_one

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.8000,0.021739,0.000000,0.032258,0.067797,0.102564,0.598684,0.017857,0.068182,0.087379
1,0.0750,0.913043,0.015873,0.032258,0.016949,0.833333,0.032895,0.892857,0.075758,0.766990
2,0.0625,0.000000,0.063492,0.919355,0.847458,0.051282,0.046053,0.041667,0.060606,0.067961
3,0.0625,0.065217,0.920635,0.016129,0.067797,0.012821,0.322368,0.047619,0.795455,0.077670
포착군집,0.0000,1.000000,3.000000,2.000000,2.000000,1.000000,0.000000,1.000000,3.000000,1.000000


In [31]:
eval_two = eval_cluster_result(model_save_together, raw_data = scaled_together, label_data=label_data_pp2)
eval_two

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.92,0.111111,0.034483,0.000000,0.153846,0.130435,0.528571,0.010753,0.046512,0.100000
1,0.00,0.722222,0.034483,0.000000,0.000000,0.826087,0.014286,0.870968,0.069767,0.750000
2,0.08,0.000000,0.000000,0.952381,0.769231,0.000000,0.100000,0.053763,0.093023,0.066667
3,0.00,0.166667,0.931034,0.047619,0.076923,0.043478,0.357143,0.064516,0.790698,0.083333
포착군집,0.00,1.000000,3.000000,2.000000,2.000000,1.000000,0.000000,1.000000,3.000000,1.000000


In [32]:
# 레이블 두개를 전부 사용한 경우의 평균 확률 -> 아예 label_data를 더해서 확률을 구하는 방법도 있음
eval_avg = (eval_one.iloc[:-1,:] + eval_two.iloc[:-1,:]) / 2
eval_avg.loc["포착군집", :] = np.argmax(eval_avg.values, axis=0)

eval_avg

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.86000,0.066425,0.017241,0.016129,0.110821,0.116499,0.563628,0.014305,0.057347,0.093689
1,0.03750,0.817633,0.025178,0.016129,0.008475,0.829710,0.023590,0.881912,0.072763,0.758495
2,0.07125,0.000000,0.031746,0.935868,0.808344,0.025641,0.073026,0.047715,0.076815,0.067314
3,0.03125,0.115942,0.925835,0.031874,0.072360,0.028149,0.339756,0.056068,0.793076,0.080502
포착군집,0.00000,1.000000,3.000000,2.000000,2.000000,1.000000,0.000000,1.000000,3.000000,1.000000


In [33]:
# 횡령인 경우
eval_three = eval_cluster_result(model_save_together, raw_data = scaled_together, label_data=label_data_pp3)
eval_three

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.625,0.071429,0.076923,0.086957,0.304348,0.28125,0.588235,0.057692,0.116279,0.041667
1,0.125,0.642857,0.076923,0.173913,0.130435,0.59375,0.078431,0.865385,0.093023,0.708333
2,0.125,0.071429,0.076923,0.521739,0.521739,0.03125,0.058824,0.057692,0.093023,0.083333
3,0.125,0.214286,0.769231,0.217391,0.043478,0.09375,0.274510,0.019231,0.697674,0.166667
포착군집,0.000,1.000000,3.000000,2.000000,2.000000,1.00000,0.000000,1.000000,3.000000,1.000000


In [34]:
# 일치 확률 평가
idx = eval_one.columns
data = np.c_[eval_one.loc["포착군집",:].values, eval_two.loc["포착군집",:].values, eval_three.loc["포착군집",:].values]

# 어디서 일차하지 않는 지 한눈에 확인 가능
df = pd.DataFrame(data=data, index=idx, columns=["라벨1","라벨2","라벨3"])


con1 = (df["라벨1"] == df["라벨2"])
con2 = (df["라벨3"] == df["라벨1"])

# 성능지표
np.mean(con1 == con2)

1.0

In [35]:
# 고위험 클러스터의 평균 확률 구하기
pro1 = eval_one.iloc[0:-1,:].max(axis=0).mean()
pro2 = eval_two.iloc[0:-1,:].max(axis=0).mean()
pro3 = eval_three.iloc[0:-1,:].max(axis=0).mean()

np.c_[pro1,pro2,pro3].mean()

0.7627648816865044

In [68]:
model_save_together2 = {}
return_masked_data = {}

for t, num in eval_avg.iloc[-1,:].iteritems():
    num = int(num)
    mask = model_save_together[t]["model"].labels_ == num
    masked_data = scaled_together[t][mask]
    
    cluster_model = MyKmeans(masked_data)
    cluster_model.set_params()
    #cluster_model.find_optimal_k(max_k=10, max_sample=10, optimize_method="silhouette")
    cluster_model.find_optimal_initp(num_of_cluster= num_k, max_sample=50, optimize_method="silhouette")
    k_mean_dict = cluster_model.run_kmean(num_of_cluster=num_k)
    return_masked_data[t] = masked_data
    model_save_together2[t] = k_mean_dict

100%|██████████| 50/50 [00:00<00:00, 71.42it/s]


In [69]:
eval_cluster_size(model_save_together2)#.max(axis=1)

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
3,141.0,98.0,96.0,96.0,142.0,113.0,72.0,117.0,153.0,169.0
0,118.0,112.0,121.0,105.0,97.0,101.0,128.0,139.0,99.0,136.0
2,111.0,83.0,118.0,109.0,100.0,147.0,146.0,133.0,128.0,171.0
1,93.0,115.0,108.0,98.0,135.0,120.0,111.0,121.0,154.0,130.0
기업수,463.0,408.0,443.0,408.0,474.0,481.0,457.0,510.0,534.0,606.0


In [71]:
prob_df1 = eval_cluster_result(model_save_together2, raw_data = scaled_together, label_data=label_data_pp1)
prob_df1

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.2625,0.304348,0.285714,0.435484,0.016949,0.243590,0.072368,0.291667,0.212121,0.349515
1,0.3750,0.521739,0.365079,0.112903,0.355932,0.141026,0.651316,0.303571,0.242424,0.126214
2,0.1750,0.086957,0.253968,0.225806,0.508475,0.243590,0.157895,0.285714,0.318182,0.262136
3,0.1875,0.086957,0.095238,0.225806,0.118644,0.371795,0.118421,0.119048,0.227273,0.262136
포착군집,1.0000,1.000000,1.000000,0.000000,2.000000,3.000000,1.000000,1.000000,2.000000,0.000000


In [72]:
prob_df2 = eval_cluster_result(model_save_together2, raw_data = scaled_together, label_data=label_data_pp2)
prob_df2

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.24,0.333333,0.275862,0.380952,0.025641,0.260870,0.042857,0.376344,0.209302,0.316667
1,0.44,0.444444,0.137931,0.190476,0.358974,0.108696,0.671429,0.236559,0.255814,0.150000
2,0.20,0.166667,0.517241,0.238095,0.435897,0.239130,0.157143,0.236559,0.348837,0.316667
3,0.12,0.055556,0.068966,0.190476,0.179487,0.391304,0.128571,0.150538,0.186047,0.216667
포착군집,1.00,1.000000,2.000000,0.000000,2.000000,3.000000,1.000000,0.000000,2.000000,0.000000


In [73]:
# 레이블 두개를 전부 사용한 경우의 평균 확률 -> 아예 label_data를 더해서 확률을 구하는 방법도 있음
eval_avg2 = (prob_df1.iloc[:-1,:] + prob_df2.iloc[:-1,:]) / 2
eval_avg2.loc["포착군집", :] = np.argmax(eval_avg2.values, axis=0)

eval_avg2

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.25125,0.318841,0.280788,0.408218,0.021295,0.252230,0.057613,0.334005,0.210712,0.333091
1,0.40750,0.483092,0.251505,0.151690,0.357453,0.124861,0.661372,0.270065,0.249119,0.138107
2,0.18750,0.126812,0.385605,0.231951,0.472186,0.241360,0.157519,0.261137,0.333510,0.289401
3,0.15375,0.071256,0.082102,0.208141,0.149066,0.381550,0.123496,0.134793,0.206660,0.239401
포착군집,1.00000,1.000000,2.000000,0.000000,2.000000,3.000000,1.000000,0.000000,2.000000,0.000000


In [128]:
eval_cluster_distance(eval_avg2, return_masked_data).sort_values("2012-01", axis=1, ascending=False)#.to_csv("1호_4_Y_거리.csv")

예측하는 거 만들기

사용할 기간을 주면 -> 이후 가능한 끝까지의 값을 예측하고, 결과를 리턴하기

In [116]:
for i in pd.date_range("2019-01", "2022-01",freq=freq+"S"):
    pd.Period(str(i)[:7])
    print(i)

2019-01-01 00:00:00
2020-01-01 00:00:00
2021-01-01 00:00:00
2022-01-01 00:00:00


In [127]:
# 이전의 Model로 이후의 데이터 클러스터링을 평가
def predict_cluster_result(model_save, raw_data, label_data,criterion_period:str, freq):
    if freq == 'Y':
        plus = 12
    elif freq == "Q":
        plus = 3
    date_idx = pd.date_range(criterion_period, "2020-01",freq=freq+'S')
    return_df = pd.DataFrame()
    for date in date_idx:
        period = pd.Period(str(date)[:7]) + plus
        date = str(date)[:7]
        eval = Optimizer.eval(model_save_together[date]["model"], scaled_together[str(period)], label_data_pp1[str(period)])
        df = pd.DataFrame(eval.values(), eval.keys(), columns=[str(period)+"예측"])
        return_df = pd.concat([return_df,df], axis=1)
    return return_df

predict_cluster_result(model_save_together, scaled_together,label_data_pp1, "2015-01", freq="Y")

,2016-01예측,2017-01예측,2018-01예측,2019-01예측,2020-01예측,2021-01예측
0,0.067797,0.089744,0.059211,0.732143,0.060606,0.097087
1,0.050847,0.051282,0.835526,0.017857,0.795455,0.077670
2,0.847458,0.833333,0.046053,0.059524,0.068182,0.077670
3,0.033898,0.025641,0.059211,0.190476,0.075758,0.747573


In [126]:
eval_one

,2012-01,2013-01,2014-01,2015-01,2016-01,2017-01,2018-01,2019-01,2020-01,2021-01
0,0.8000,0.021739,0.000000,0.032258,0.067797,0.102564,0.598684,0.017857,0.068182,0.087379
1,0.0750,0.913043,0.015873,0.032258,0.016949,0.833333,0.032895,0.892857,0.075758,0.766990
2,0.0625,0.000000,0.063492,0.919355,0.847458,0.051282,0.046053,0.041667,0.060606,0.067961
3,0.0625,0.065217,0.920635,0.016129,0.067797,0.012821,0.322368,0.047619,0.795455,0.077670
포착군집,0.0000,1.000000,3.000000,2.000000,2.000000,1.000000,0.000000,1.000000,3.000000,1.000000


In [ ]:
pd.DataFrame(model_save_together["2021-01"]["model"].predict(scaled_together["2022-07"])).value_counts()

### 코스닥 평가

In [ ]:
eval_cluster_size(model_save_kosdaq)

In [ ]:
eval_cluster_result(model_save_kosdaq, raw_data = scaled_kosdaq, label_data=label_data_pp1, freq='Y')

In [ ]:
eval_cluster_result(model_save_kosdaq, raw_data = scaled_kosdaq, label_data=label_data_pp2, freq='Y')

In [ ]:
eval_cluster_result(model_save_kosdaq, raw_data = scaled_kosdaq, label_data=label_data_pp3, freq='Y')

### 코스피결과

In [ ]:
eval_cluster_size(model_save_kospi)

In [ ]:
eval_cluster_result(model_save_kospi, raw_data = scaled_kospi, label_data=label_data_pp1, freq='Y')